In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

In [ ]:
#Start Spark session
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
spark=SparkSession.builder.appName("Tokenize Data").getOrCreate()


In [ ]:
#tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_65b4e0e330c0

In [ ]:
#test dataframe
df = spark.createDataFrame([
    (0, "hello world"),
    (1, "greetings, earth"),
    (2, "waddup, third rock from the sun")
], ["id", "sentence"])

In [ ]:
toked_df = tokenizer.transform(df)
toked_df.show(truncate=False)

+---+-------------------------------+--------------------------------------+
|id |sentence                       |words                                 |
+---+-------------------------------+--------------------------------------+
|0  |hello world                    |[hello, world]                        |
|1  |greetings, earth               |[greetings,, earth]                   |
|2  |waddup, third rock from the sun|[waddup,, third, rock, from, the, sun]|
+---+-------------------------------+--------------------------------------+



In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
  return len(word_list)

In [ ]:
#to employ udf defined above import these spark libraries
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

we can redo the `tokenizer` process. Only this time, after the DataFrame has outputted the tokenized values, we can use our own created function to return the number of tokens created. This will give us another data point to use in the future, if needed.

In [ ]:
#create tokenizer
tokenizer=Tokenizer(inputCol="sentence", outputCol="words")

#transform df
toked_df = tokenizer.transform(df)

#select needed columns and don't truncate results
toked_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)



+---+-------------------------------+--------------------------------------+------+
|id |sentence                       |words                                 |tokens|
+---+-------------------------------+--------------------------------------+------+
|0  |hello world                    |[hello, world]                        |2     |
|1  |greetings, earth               |[greetings,, earth]                   |2     |
|2  |waddup, third rock from the sun|[waddup,, third, rock, from, the, sun]|6     |
+---+-------------------------------+--------------------------------------+------+

